In [18]:
import sys
import bs4
import traceback
import re
import time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome import service as fs

In [19]:
#ドライバーのフルパス
CHROMEDRIVER = "C:\\Users\\ayumi\\chromedr\\chromedriver.exe"
chrome_service = fs.Service(executable_path=CHROMEDRIVER)
#改ページ
PAGE_MAX = 2
#遷移間隔
Interval_time = 3
#開始年、月、日
SY = 2020
SM = 1
SD = 1
#終了月
EY = 2022
EM = 12
ED = 31
#タイプ(d:デイリー、w:週刊、m:月間)
TM = 'm'

In [20]:
#ドライバー準備
def get_driver():
    #ヘッドレスモードでブラウザ起動
    options = Options()
    options.add_argument('--headless')
    
    #ブラウザを起動
    driver = webdriver.Chrome(service=chrome_service, options=options)
    
    return driver

#対象ページのソース取得
def get_source_from_page(driver, page):
    try:
        driver.get(page)
        driver.implicitly_wait(10)
        page_source = driver.page_source
        
        return page_source
    except Exception as e:
        print('Exception\n' + traceback.format_exc())
        return None
    
def get_data_from_source(src):
    #スクレイピングする
    soup = bs4.BeautifulSoup(src, features='lxml')
    #print(soup)
    try:
        info = []
        table = soup.find('table', class_='boardFin')
        
        if table:
            elems = table.find_all('tr')
            
            for elem in elems:
                td_tags = elem.find_all('td')
                
                if len(td_tags) > 0:
                    row_info = []
                    tmp_counter = 0
                    for td_tag in td_tags:
                        tmp_text = td_tag.text
                        
                        if tmp_counter == 0:
                            tmp_text = tmp_text
                        else:
                            tmp_text = extract_num(tmp_text)
                        
                        row_inf.append(tmp_text)
                        tmp_counter += 1
                    info.append(row_info)
        return info
    except Exception as e:
        print('Exception\n' + traceback.format_exc())
        return None
    
#次のページへ遷移
def next_btn_click(driver):
    try:
        #次へボタン
        elem_btn = WebDriverWait(driver, 10).until(EC.visibility_of_all_elements_located((By.LINK_TEXT, '次へ')))
        #クリック処理
        actions = ActionChains(driver)
        actions.move_to_element(elem_btn)
        actions.click(elem_btn)
        actions.perform()
        
        #間隔を設ける（秒単位）
        time.sleep(Interval_time)
        return True
    except Exception as e:
        print('Exception\n' + traceback.format_exc())
        return False

#数値だけ抽出
def extract_num(val):
    num = None
    if val:
        match = re.findall('\d+\.\d+', val)
        if len(match) > 0:
            num = match[0]
        else:
            num = re.sub('\\D', '', val)

In [21]:
if __name__ == '__main__':
    args = sys.argv
    #銘柄コード
    code = '9301'
    if len(args) == 2:
        code = args[1]
        
    #対象ページURL
    page = 'https://info.finance.yahoo.co.jp/history/?code=' + code
    page = page + '&amp;sy='+str(SY) + '&amp;sm='+str(SM) + '&amp;sd='+str(SD)
    page = page + '&amp;ey='+str(EY) + '&amp;em='+str(EM) + '&amp;ed='+str(ED)
    page = page + '&amp;tm='+TM
        
    #ブラウザのドライバー取得
    driver = get_driver()
        
    #ページのソース取得
    source = get_source_from_page(driver, page)
    result_flg = True
        
    #ページカウンター制御
    page_counter = 0
        
    while result_flg:
        page_counter += 1
        #ソースからデータ抽出
        data = get_data_from_source(source)
            
        #データ保存
        print(data)
            
        #改ページ処理を抜ける
        if page_counter == PAGE_MAX:
            break
            
        #改ページ処理
        result_flg = next_btn_click(driver)
        source = driver.page_source
        
    #閉じる
    driver.quit()

[]
Exception
Traceback (most recent call last):
  File "C:\Users\ayumi\AppData\Local\Temp\ipykernel_9084\1659058492.py", line 61, in next_btn_click
    elem_btn = WebDriverWait(driver, 10).until(EC.visibility_of_all_elements_located((By.LINK_TEXT, '次へ')))
  File "C:\Users\ayumi\anaconda3\lib\site-packages\selenium\webdriver\support\wait.py", line 95, in until
    raise TimeoutException(message, screen, stacktrace)
selenium.common.exceptions.TimeoutException: Message: 


